<a href="https://colab.research.google.com/github/Rohitsingh24-cloud/pytorch/blob/main/optimizedANNusingpytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
from  sklearn.model_selection import train_test_split

In [3]:
import pandas as pd

In [4]:
from torch.utils.data import DataLoader,Dataset

In [5]:
import torch.nn as nn

In [6]:
import torch.optim as optim

In [7]:
import matplotlib.pyplot as plt



In [8]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [9]:
torch.manual_seed(42)

In [10]:
df=pd.read_csv('fashion-mnist_train.csv')

In [11]:
df

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#train test split

In [13]:
x=df.iloc[:,1:].values

In [14]:
y=df.iloc[:,0].values

In [15]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [16]:
x_train=x_train/255.0
x_test=x_test/255.0

In [17]:
#create custom dataset class

In [18]:
class customdataset(Dataset):
  def __init__(self,features,lables):
    self.features=torch.tensor(features,dtype=torch.float32)
    self.labels=torch.tensor(lables,dtype=torch.long)
  def __len__(self):
    return len(self.features)
  def __getitem__(self,idx):
    x=self.features[idx]
    y=self.labels[idx]
    return x,y

In [19]:
train_dataset=customdataset(x_train,y_train)

In [20]:
test_dataset=customdataset(x_test,y_test)

In [21]:
#crate train and test loader

In [22]:
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True,pin_memory=True,num_workers=2)


In [23]:
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False,pin_memory=True,num_workers=2)

In [24]:
##define nn class

In [25]:
class MyNN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.model=nn.Sequential(
            nn.Linear(num_features,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(64,10)
        )

    def forward(self,x):
        return self.model(x)


In [26]:
learning_rate=0.1
epochs=100

In [27]:
model=MyNN(x_train.shape[1])
model=model.to(device)

In [28]:
#loss function
criterion=nn.CrossEntropyLoss()

In [29]:
#optimizer
optimizer=optim.SGD(model.parameters(),lr=learning_rate,weight_decay=1e-4)

In [30]:
#train loop

In [31]:
for epoch in range(epochs):
  total_epoch_loss=0;

  for batch_features,batch_labels in train_loader:
    #move data to gpu
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)
    #forward pass
    outputs=model(batch_features)
    #calculate loss
    Loss=criterion(outputs,batch_labels)
    #back propagate
    optimizer.zero_grad()
    Loss.backward()

    #update gradient
    optimizer.step()
    total_epoch_loss=total_epoch_loss+Loss.item()
  avg_loss=total_epoch_loss/len(train_loader)
  print(f'Epoch:{epoch+1},Loss:{avg_loss}')

Epoch:1,Loss:0.6249084644317627
Epoch:2,Loss:0.4919969036479791
Epoch:3,Loss:0.455620892132322
Epoch:4,Loss:0.4338064221441746
Epoch:5,Loss:0.41715061252812546
Epoch:6,Loss:0.4056409275531769
Epoch:7,Loss:0.39416088837385177
Epoch:8,Loss:0.3858017429014047
Epoch:9,Loss:0.37439838365713757
Epoch:10,Loss:0.37257265872259937
Epoch:11,Loss:0.36783314737677575
Epoch:12,Loss:0.35720522227386636
Epoch:13,Loss:0.3505206037561099
Epoch:14,Loss:0.3449219425916672
Epoch:15,Loss:0.34472561853379013
Epoch:16,Loss:0.337321236188213
Epoch:17,Loss:0.3344038932522138
Epoch:18,Loss:0.3302020431111256
Epoch:19,Loss:0.33063985271751883
Epoch:20,Loss:0.3262277199824651
Epoch:21,Loss:0.32084967653950053
Epoch:22,Loss:0.3183093220243851
Epoch:23,Loss:0.3225850307941437
Epoch:24,Loss:0.31459669538835683
Epoch:25,Loss:0.3134305283402403
Epoch:26,Loss:0.31424527227630217
Epoch:27,Loss:0.3107087374528249
Epoch:28,Loss:0.3090560559928417
Epoch:29,Loss:0.3045613792786996
Epoch:30,Loss:0.3055545740922292
Epoch:31,L

In [32]:
#set model to eval

In [33]:
model.eval()
#it tell model that we are going for evaluation

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [34]:
total=0
correct=0;
with torch.no_grad():
  for batch_features,batch_labels in test_loader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)
    outputs=model(batch_features)
    _,predicted=torch.max(outputs,1)
    total=total+batch_labels.shape[0]
    correct=correct+(predicted==batch_labels).sum().item()
print(correct/total)

0.88325
